In [7]:
% matplotlib inline
import numpy as np
from helpers import load_data
import scipy.sparse as sp

# trained on whole dataset:
#'../submission/train_surprise_unrounded_0.99256.csv'
files_full_submission = [ \
    'training_prediction_ALS_BaselineOnly.csv',\
    'training_prediction_KNNBasic_ALS_pearson_baseline_item_based.csv',\
    'training_prediction_KNNBasic_ALS_pearson_baseline_user_based.csv',\
    'training_prediction_KNNBasic_pearson_baseline_user_based.csv',\
    'training_prediction_SGD_BaselineOnly.csv'
    ]
files_submission = [ \
    'submission_ALS_BaselineOnly.csv',\
    'submission_KNNBasic_ALS_pearson_baseline_item_based.csv',\
    'submission_KNNBasic_ALS_pearson_baseline_user_based.csv',\
    'submission_KNNBasic_pearson_baseline_user_based.csv',\
    'submission_SGD_BaselineOnly.csv'
                    ]
#files_submission = [ \
#        '../submission/submission_surprise_unrounded_0.99256.csv']

files_full_numpy = [\
        #'../submission/saved_0.98475.npy',\
        #'../submission/saved_0.984.npy', \
        '../saved/blend_0.06255_8.npy']
#errors_submission = [0.98475, 0.984, 0.98431, 0.99256]

names = ['ALS', 'ALS_Baseline', 'KNN_ALS_item', 'KNN_ALS_user', 'KNN_user', 'SGD_Baseline']
#names = ['ALS','ALS','ALS','SVD surprise']
file_train_true = '../data/data_train.csv'
file_submission_true = '../data/sampleSubmission.csv'

In [5]:
from blending import get_all_indices, read_numpy_files, apply_indices
from our_helpers import load_data
# Read full matrices 
train_true = load_data(file_train_true)
training_true = apply_indices([train_true], 'train')[0]
test_true = apply_indices([train_true], 'test')[0]
validation_true = apply_indices([train_true], 'validation')[0]

number of items: 10000, number of users: 1000
number of items: 10000, number of users: 1000
number of rows, cols: 1117939
number of non-zero elements in matrix 0: 1117939
number of items: 10000, number of users: 1000
number of rows, cols: 29403
number of non-zero elements in matrix 0: 29403
number of items: 10000, number of users: 1000
number of rows, cols: 29610
number of non-zero elements in matrix 0: 29610


In [8]:
submission_true = load_data(file_submission_true)
indices = get_all_indices(train_true, submission_true)
trains_full = read_numpy_files(files_full_numpy, indices)

number of items: 10000, number of users: 1000
number of rows, cols to read out: 2353904
matrix 0
dense matrix shape: (10000, 1000)
number of non-zero elements in matrix: 2353904


In [9]:
# Apply indices.
from blending import apply_indices
train_est = apply_indices(trains_full, 'train') 
test_est = apply_indices(trains_full, 'test') 
submission_est = apply_indices(trains_full, 'submission') 
validation_est = apply_indices(trains_full, 'validation') 

number of items: 10000, number of users: 1000
number of rows, cols: 1117939
number of non-zero elements in matrix 0: 1117939
number of items: 10000, number of users: 1000
number of rows, cols: 29403
number of non-zero elements in matrix 0: 29403
number of items: 10000, number of users: 1000
number of rows, cols: 1176952
number of non-zero elements in matrix 0: 1176952
number of items: 10000, number of users: 1000
number of rows, cols: 29610
number of non-zero elements in matrix 0: 29610


In [ ]:
# Add matrices saved in submission format.
from blending import read_submission_files
surprise_full = read_submission_files(files_full_submission)
surprise_train_est = apply_indices(surprise_full, 'train') 
surprise_test_est = apply_indices(surprise_full, 'test') 
surprise_validation_est = apply_indices(surprise_full, 'validation') 


surprise_submission = read_submission_files(files_submission)

for i in range(len(surprise_train_est)):
    validation_est.append(surprise_validation_est[i])
    train_est.append(surprise_train_est[i])
    test_est.append(surprise_test_est[i])
    submission_est.append(surprise_submission[i])

In [ ]:
from data_postprocess import create_sparse_matrix_plot
# Visualize matrices
i_total, ratings_dense = create_sparse_matrix_plot(training_true, '../results/Blending/matrix_train.png')

dense_matrices=[]
dense_matrices.append(ratings_dense.copy())
#j_total, _ = create_matrix_plot(train_true, axis=1)
for i,matrix_est in enumerate(train_est):
    __, ratings_dense = create_sparse_matrix_plot(matrix_est, '../results/Blending/matrix_{}'.format(i+1))
    dense_matrices.append(ratings_dense.copy())

In [ ]:
for i in range(1,len(dense_matrices)):
    diff_matrix = dense_matrices[i]-dense_matrices[0]
    print('diff goes from {} to {}'.format(np.max(diff_matrix),np.min(diff_matrix)))
    matrix_plot(diff_matrix, '../results/matrix_diff{}.png'.format(i), 1000)

In [ ]:
from blending import linear_blending
q_hat, x = linear_blending(test_est, submission_est, test_true)
print(q_hat)
print(len(q_hat))

In [ ]:
cutoff = 1000
rows,cols,true_values = sp.find(validation_true)
file_name = '../results/Blending/matrix_validation.png'
title = 'Validation data'
i_total, validation_dense = create_sparse_matrix_plot(validation_true, file_name ,cutoff, title)
Q = np.empty((len(true_values),len(validation_est)))
for i,validation_prediction in enumerate(validation_est):
    __,__,predictions = sp.find(validation_prediction)
    Q[:,i] = predictions
    rmse = np.sqrt(np.sum(np.power(predictions-true_values, 2)) / len(true_values))
    print('rmse method {}: {}'.format(i,rmse))
    file_name = '../results/Blending/matrix_validation_{}.png'.format(i)
    title = 'Method {}, validation error: {:1.5f}, test error: {:1.5f}'.format(names[i], rmse, errors_submission[i])
    create_sparse_matrix_plot(validation_prediction, file_name, cutoff, title)
    print(predictions - true_values)
blending_prediction = np.dot(Q, x) 
rmse = np.sqrt(np.sum(np.power(blending_prediction-true_values, 2)) / len(true_values))
print('blending method: {}'.format(i,rmse))

# create sparse matrix from predictions
validation_blending = sp.lil_matrix(validation_true.shape)
for k, (i,j) in enumerate(zip(rows, cols)):
    validation_blending[i,j] = blending_prediction[k]
file_name = '../results/Blending/matrix_blending.png'
title = 'Method Blending, validation error: {:1.5f}'.format(rmse)
test = create_sparse_matrix_plot(validation_blending, file_name, cutoff, title)